In [1]:
import warnings
warnings.filterwarnings("ignore")
from ipywidgets import  Image, VBox, Label, HBox

In [2]:

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, Input, BatchNormalization, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [3]:
from matplotlib import pyplot as plt1

In [4]:
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2

In [5]:
from keras import backend as K 
classifier = Sequential() 

if K.image_data_format() == 'channels_first': 
    input_shape = (3, 720, 1280) 
else: 
    input_shape = (720, 1280, 3) 


In [337]:
K.clear_session()

In [6]:
input_shape = (720,720,3)

In [267]:
inp =  Input(shape=input_shape)

In [268]:
base_model = InceptionResNetV2(input_tensor = inp, weights='imagenet', include_top = False)

In [269]:
for layer in base_model.layers:
    layer.trainable = False

In [270]:
cls.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 720, 720, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 359, 359, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 359, 359, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 359, 359, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

batch_normalization_193 (BatchN (None, 21, 21, 192)  576         conv2d_193[0][0]                 
__________________________________________________________________________________________________
activation_193 (Activation)     (None, 21, 21, 192)  0           batch_normalization_193[0][0]    
__________________________________________________________________________________________________
conv2d_194 (Conv2D)             (None, 21, 21, 224)  129024      activation_193[0][0]             
__________________________________________________________________________________________________
batch_normalization_194 (BatchN (None, 21, 21, 224)  672         conv2d_194[0][0]                 
__________________________________________________________________________________________________
activation_194 (Activation)     (None, 21, 21, 224)  0           batch_normalization_194[0][0]    
__________________________________________________________________________________________________
conv2d_192

In [271]:
model = GlobalAveragePooling2D()(base_model.output)
model = Dense(100, activation='tanh')(model)
model = BatchNormalization()(model)
model = Dropout(0.1)(model)
otpt = Dense(2, activation='sigmoid')(model)

In [272]:
cls = Model(input=base_model.input, output=otpt)
cls.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
cls.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 720, 720, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 359, 359, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 359, 359, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 359, 359, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [7]:
cls = load_model("inception_based.h5")

In [8]:
import pandas as pd
data = pd.read_csv('train_names.csv')
data.columns=['filename']

In [306]:
cnt = 0

for r in data.itertuples():
#    if str(r).__contains__('_CYY_') or str(r).__contains__('_CNY_') or str(r).__contains__('_CNN_') or str(r).__contains__('_CYN_'):
#        continue
#    elif str(r).__contains__('_C_'):
 #       continue
#    elif str(r).__contains__('_C_'):
 #       continue
 #   else :
   #     print(str(r))
    if bool(f.findall(str(r))):
        cnt += 1
print(cnt) 

1950


In [9]:
def filterData(d):
    import re
    f = re.compile("C[N+Y][N+Y]")
    fd = []
    for r in d.values.tolist():
        if bool(f.findall(str(r))):
            fd.append(r)
    fltrdata = pd.DataFrame(fd, columns = d.columns)
    return fltrdata

In [10]:
data = filterData(data)

In [11]:
import re
fltr = re.compile("C[A-Z]Y")
print(fltr.findall('DE_8426944020409_NPPFN_CCY_C_20170502165734861.jpg'))

['CCY']


In [12]:
data['label'] = data.applymap(lambda x: 1 if len(fltr.findall(x)) > 0 else 0)

In [17]:
data.sample(10)

,filename,label
266,EE_0885190821242_00000_U_CNN_20190404121652911...,0
7329,NL_8718166030435_00000_U_CNN_20150129105406005...,0
4900,NL_8710400433774_BBNEW_C100_CYN_20160413-01.jpg,0
7827,NL_8718452368860_BBREV_C100_CYY_20190520.jpg,1
6828,NL_8716528977213_BBREV_C100_CYY_20190507.jpg,1
2664,NL_4008233047157_00000_C100_CNN_20120405161436...,0
7106,NL_8717399035224_BBNEW_C100_CYY_20151111.jpg,1
4151,NL_8001841190990_BBNEW_C100_CYN_20190521-00.jpg,0
10132,ua_4015400552031_00000_U_CNY_20131205104402110...,1
486,EE_4005808656738_00000_C100_CNN_20120626134204...,0


In [18]:
from sklearn.model_selection import train_test_split

In [19]:
train_data, test_data = train_test_split(data, test_size=0.05, stratify=data['label'])

In [20]:
test_data.groupby('label').count()

,filename
label,
0,388
1,123


In [21]:
from sklearn.utils import shuffle

In [22]:
def train_iterator(data, batch_size=32):
    while True:        
        positive = data.query("label == 1").sample(batch_size//2)
        negative = data.query("label == 0").sample(batch_size//2)
        train_files = pd.concat([positive,negative]).values
        features = []
        labels = []
        for file in train_files:
            img = image.load_img('D:/OGRDS/Image Classification/Train/'+file[0], target_size=input_shape)
            img = image.img_to_array(img)
            img = img/255
            features.append(img)
        yield (np.array(features), np.array([[1,0] if x[1]==0 else [0,1] for x in train_files]))                

In [29]:
from keras.optimizers import Adam
optimizer = Adam(lr=0.0001)

In [26]:
Sequence?

In [39]:
def test_iterator(data,dir_name='Train', batch_size=32):
    train_files = data.values
    i = 0
    features = []
    for file in train_files:
        yield preprocess_image(file[0], dir_name)

In [30]:
cls.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [31]:
for i in range(4):
    batch_size = 32
    cls.fit_generator(
             train_iterator(train_data), 
             steps_per_epoch=100,
             epochs=3)
    cls.save("inception_based.h5")

Epoch 1/3
100/100 [==============================] - 282s 3s/step - loss: 0.0692 - acc: 0.9747
Epoch 2/3
100/100 [==============================] - 282s 3s/step - loss: 0.0686 - acc: 0.9753
Epoch 3/3
100/100 [==============================] - 281s 3s/step - loss: 0.0602 - acc: 0.9784
Epoch 1/3
100/100 [==============================] - 281s 3s/step - loss: 0.0635 - acc: 0.9766
Epoch 2/3
100/100 [==============================] - 280s 3s/step - loss: 0.0563 - acc: 0.9806
Epoch 3/3
100/100 [==============================] - 280s 3s/step - loss: 0.0589 - acc: 0.9819
Epoch 1/3
100/100 [==============================] - 281s 3s/step - loss: 0.0615 - acc: 0.9769
Epoch 2/3
100/100 [==============================] - 280s 3s/step - loss: 0.0511 - acc: 0.9814
Epoch 3/3
100/100 [==============================] - 280s 3s/step - loss: 0.0519 - acc: 0.9819
Epoch 1/3
100/100 [==============================] - 281s 3s/step - loss: 0.0542 - acc: 0.9767
Epoch 2/3
100/100 [==============================]

In [88]:
def get_image(img_file, dir_name='Test'):
    with open('D:/OGRDS/Image Classification/{}/'.format(dir_name)+img_file, 'rb') as f:
        return f.read()

In [89]:
def preprocess_image(img_file, dir_name='Test'):
    img = image.load_img('D:/OGRDS/Image Classification/{}/'.format(dir_name)+img_file, target_size=input_shape)
    img = image.img_to_array(img)
    img = img/255
    return np.array([img])

In [90]:
feature = preprocess_image('AT_5449000000279_00000_U_CNN_20130823032503071.jpg')

np.argmax(cls.predict(feature))

0

In [95]:
testData = pd.read_csv('test_names.csv')
testData.columns=['filename']
testData = filterData(testData)
testData['actual_label'] = testData.applymap(lambda x: 1 if len(fltr.findall(x)) > 0 else 0)

,filename
actual_label,
0,1335
1,615


In [100]:
for rows in testData.values:
    feature = preprocess_image(rows[0])

['AT_0000054024892_00000_C100_CNN_20130917024756495.JPG' 0] 0
['AT_0000054024892_00000_C100_CNN_20130917034050325.JPG' 0] 0
['AT_0000054024892_00000_C100_CNN_20130917034053231.JPG' 0] 0
['AT_0000054024892_00000_C100_CNN_20130917034101528.JPG' 0] 0
['AT_0000054024892_00000_C100_CNN_20130917034104185.JPG' 0] 0
['AT_0000054024892_00000_C100_CNN_20171128142008741.JPG' 0] 0
['AT_0000054024892_00000_C100_CNN_20171128142011950.JPG' 0] 0
['AT_0000054024892_00000_U_CNN_20130917024759714.JPG' 0] 0
['AT_0000054024892_00000_U_CNN_20130917024806574.JPG' 0] 0
['AT_0000054024892_00000_U_CNN_20130917024812933.JPG' 0] 1
['AT_0000054024892_00000_U_CNN_20130917034056091.JPG' 0] 0
['AT_0000054024892_00000_U_CNN_20130917034058841.JPG' 0] 1
['AT_0000054024892_00000_U_CNN_20130917034106841.JPG' 0] 1
['AT_0000054024892_00000_U_CNN_20130917034109591.JPG' 0] 1
['AT_0000054024892_00000_U_CNN_20171128142015377.JPG' 0] 0
['AT_0000075032814_00000_C100_CNN_20100519103344625.JPG' 0] 0
['AT_0000075032814_00000_U_CNN_2

['BE_0000020092276_00000_U_CNN_20110721080340640.JPG' 0] 1
['BE_0000020092276_00000_U_CNN_20110721080346281.JPG' 0] 1
['BE_0000020092276_00000_U_CNN_20110721080354687.JPG' 0] 0
['BE_0000020092276_00000_U_CNN_20110721080400062.JPG' 0] 1
['BE_0000020092276_00000_U_CNY_20110721080335124.JPG' 1] 1
['BE_0000020092276_00006_U_CNN_20150611101256931.JPG' 0] 1
['BE_0000020092276_00006_U_CNN_20150611101658572.JPG' 0] 1
['BE_0000020092276_00006_U_CNN_20150611101802494.JPG' 0] 0
['BE_0000020092276_00006_U_CNN_20150611102055041.JPG' 0] 0
['BE_0000020092276_00006_U_CNN_20150611102310822.JPG' 0] 1
['BE_0000020092276_00006_U_CNN_20150611102344275.JPG' 0] 1
['BE_0000020119232_00000_U_CNN_20110721080404656.JPG' 0] 0
['BE_0000020119232_00000_U_CNN_20110721080412968.JPG' 0] 1
['BE_0000020119232_00000_U_CNN_20110721080418265.JPG' 0] 1
['BE_0000020119232_00000_U_CNN_20110721080427656.JPG' 0] 0
['BE_0000020119232_00000_U_CNN_20110721080432781.JPG' 0] 1
['BE_0000020119232_00000_U_CNY_20110721080407468.JPG' 1]

['BE_0000075032814_00000_C100_CNN_20110217073149362.JPG' 0] 0
['BE_0000075032814_00000_C100_CNN_20150212010802387.JPG' 0] 0
['BE_0000075032814_00000_U_CNN_20110217073200690.JPG' 0] 0
['BE_0000075032814_00000_U_CNN_20110217073207143.JPG' 0] 0
['BE_0000075032814_00000_U_CNN_20110217073213643.JPG' 0] 1
['BE_0000075032814_00000_U_CNY_20110217073151878.JPG' 1] 1
['BE_0000075032814_00004_U_CNN_20150212012512075.JPG' 0] 0
['BE_0000075032814_00004_U_CNN_20150212015113233.JPG' 0] 0
['BE_0000075032814_00004_U_CNN_20150212024449015.JPG' 0] 0
['BE_0000075032814_00004_U_CNN_20150212031622421.JPG' 0] 0
['BE_0000075032814_00004_U_CNN_20150212040501811.JPG' 0] 0
['BE_0000075032814_00004_U_CNN_20150212060919436.JPG' 0] 0
['BE_0000075032814_00004_U_CNY_20150212025344452.JPG' 1] 1
['BE_0000075032814_BBREV_C100_CYN_20170508.jpg' 0] 0
['BE_0000075032814_BBREV_C100_CYN_20190508.jpg' 0] 1
['BE_0000075032814_BBREV_C100_CYY_20170508.jpg' 1] 1
['BE_0000075032814_BBREV_C100_CYY_20190508.jpg' 1] 1
['BE_0000075041

['BE_3574661416748_BBNEW_C100_CYN_20190426.jpg' 0] 1
['BE_3574661416748_BBNEW_C100_CYY_20190426.jpg' 1] 1
['BE_3574661419992_CLNPC_C100_CNY_20180612193400000.jpg' 1] 0
['BE_3574661419992_CLNPC_C100_CNY_20180612193400001.jpg' 1] 0
['BE_3574661422787_BBNEW_C100_CYN_20180817.jpg' 0] 0
['BE_3574661422787_CLNPC_C100_CNY_20180612193400000.jpg' 1] 0
['BE_3574661422787_CLNPC_C100_CNY_20180612193400001.jpg' 1] 0
['BE_3574661423005_BBNEW_C100_CYN_20180817.jpg' 0] 0
['BE_3574661423005_CLNPC_C100_CNY_20180612193400000.jpg' 1] 0
['BE_3574661423005_CLNPC_C100_CNY_20180612193400001.jpg' 1] 0
['BE_3574661424392_BBNEW_C100_CYN_20190426.jpg' 0] 0
['BE_3574661424392_BBNEW_C100_CYY_20190426.jpg' 1] 1
['BE_3574661497471_BBNEW_C100_CYN_20190426.jpg' 0] 1
['BE_3574661497471_BBNEW_C100_CYY_20190426.jpg' 1] 1
['BE_3574661497488_BBNEW_C100_CYN_20190426.jpg' 0] 0
['BE_3574661497488_BBNEW_C100_CYY_20190426.jpg' 1] 1
['BE_3600541524422_00000_C100_CNN_20150226045347414.JPG' 0] 0
['BE_3600541524422_00000_U_CNN_20150

['BE_4084500999831_BBREV_C100_CYN_20160928.jpg' 0] 0
['BE_4084500999831_BBREV_C100_CYN_20161007-00.jpg' 0] 0
['BE_4084500999831_BBREV_C100_CYN_20161025-00.jpg' 0] 0
['BE_4084500999831_BBREV_C100_CYN_20161228-00.jpg' 0] 0
['BE_4084500999831_BBREV_C100_CYN_20161229-00.jpg' 0] 0
['BE_4084500999831_BBREV_C100_CYY_20160928.jpg' 1] 1
['BE_4084500999831_BBREV_C100_CYY_20161007.jpg' 1] 1
['BE_4084500999831_BBREV_C100_CYY_20161025.jpg' 1] 1
['BE_4084500999831_BBREV_C100_CYY_20161228.jpg' 1] 1
['BE_4084500999831_BBREV_C100_CYY_20161229.jpg' 1] 1
['BE_4210201105305_BBNEW_C100_CYN_20161028-00.jpg' 0] 0
['BE_4210201105305_BBNEW_C100_CYY_20161028.jpg' 1] 1
['BE_4210201105305_BBREV_C100_CYN_20161028-00.jpg' 0] 0
['BE_4210201105305_BBREV_C100_CYN_20170222-00.jpg' 0] 0
['BE_4210201105305_BBREV_C100_CYN_20170223-00.jpg' 0] 0
['BE_4210201105305_BBREV_C100_CYY_20161028.jpg' 1] 1
['BE_4210201105305_BBREV_C100_CYY_20170222.jpg' 1] 1
['BE_4210201105305_BBREV_C100_CYY_20170223.jpg' 1] 1
['BE_4210201746249_BBR

...

['BE_8718182760019_00000_U_CNY_20120611081201276.JPG' 1] 1
['BE_8718182760026_00000_C100_CNN_20120425063653061.JPG' 0] 0
['BE_8718182760026_00000_U_CNN_20120425063813606.JPG' 0] 0
['BE_8718182760026_00000_U_CNN_20120425063846465.JPG' 0] 0
['BE_8718182760026_00000_U_CNN_20120425063902981.JPG' 0] 0
['BE_8718182760026_00000_U_CNN_20120425063931043.JPG' 0] 0
['BE_8718182760026_00000_U_CNY_20120425063728701.JPG' 1] 0
['BE_8718265771796_BBNEW_C100_CYN_20171215.jpg' 0] 0
['BE_8718265771796_BBNEW_C100_CYY_20171215.jpg' 1] 0
['BE_8718347282370_BBNEW_C100_CYN_20180531.jpg' 0] 0
['BE_9049000000444_BBNEW_C100_CYN_20161013.jpg' 0] 0
['BE_9049000000444_BBNEW_C100_CYY_20161013.jpg' 1] 1
['BG_0000075032814_00000_C100_CNN_20101209152316100.JPG' 0] 0
['BG_0000075032814_00000_C100_CNN_20101209152349429.JPG' 0] 0
['BG_0000075032814_00000_U_CNN_20101209152331351.JPG' 0] 0
['BG_0000075032814_00000_U_CNN_20101209152409258.JPG' 0] 0
['BG_0000075032814_00000_U_CNN_20101209152427993.JPG' 0] 0
['BG_0000075032814

['CH_0882381004712_00000_C100_CNN_20140425082907727.JPG' 0] 0
['CH_0882381004712_00000_U_CNN_20140425082910054.JPG' 0] 0
['CH_0882381004712_00000_U_CNN_20140425082914180.JPG' 0] 0
['CH_0882381004712_00000_U_CNN_20140425082919988.JPG' 0] 1
['CH_0882381051747_00000_C100_CNN_20130624014429682.JPG' 0] 0
['CH_0882381051747_00000_C100_CNN_20130624014431839.JPG' 0] 1
['CH_0882381051747_00000_U_CNN_20130624014433995.JPG' 0] 0
['CH_0882381051747_00000_U_CNN_20130624014436151.JPG' 0] 1
['CH_0882381051747_00000_U_CNN_20130624014438292.JPG' 0] 0
['CH_0882381051747_00000_U_CNN_20130624014440448.JPG' 0] 0
['CH_0882381051747_00000_U_CNN_20130624014442620.JPG' 0] 0
['CH_0882381051747_00000_U_CNN_20130624014444777.JPG' 0] 0
['CH_0882381051747_00000_U_CNN_20130624014446917.JPG' 0] 1
['CH_0882381060213_00000_C100_CNN_20140425082815812.JPG' 0] 0
['CH_0882381060213_00000_U_CNN_20140425082818154.JPG' 0] 0
['CH_0882381060213_00000_U_CNN_20140425082822453.JPG' 0] 0
['CH_3065892900002_00000_C100_CNN_2013022612

['CS_0882381060213_00000_U_CNN_20140110144553824.JPG' 0] 0
['CS_0882381060213_00000_U_CNN_20140110144556138.JPG' 0] 0
['CS_0882381060213_00000_U_CNN_20140110144558639.JPG' 0] 0
['CS_0882381069322_00000_C100_CNN_20141231083827739.JPG' 0] 0
['CS_0882381069322_00000_U_CNN_20141231083830178.JPG' 0] 0
['CS_0882381069322_00000_U_CNN_20141231083832632.JPG' 0] 1
['CS_8428390056537_00000_C080_CNN_20150831091026877.JPG' 0] 0
['CS_8428390056537_00000_U_CNN_20150831091031811.JPG' 0] 0
['CS_8428390056995_00000_C100_CNN_20150831091042782.JPG' 0] 0
['CS_8428390056995_00000_U_CNN_20150831091047172.JPG' 0] 0
['CS_8714789732886_00000_U_CNN_20120803140446238.JPG' 0] 0
['CS_8714789732893_00000_U_CNN_20120803140315797.JPG' 0] 0
['CS_8714789732893_00000_U_CNN_20120827113815617.JPG' 0] 0
['CS_8714789732893_00000_U_CNN_20120827113830244.JPG' 0] 0
['CZ_0000020203504_00000_U_CNN_20190108094619355.JPG' 0] 0
['CZ_0000020203504_00000_U_CNN_20190108094636493.JPG' 0] 0
['CZ_0000020203504_00000_U_CNY_2019010809462390

['CZ_9005800284002_BBNEW_C100_CYN_20170406.jpg' 0] 0
['CZ_9005800284002_BBNEW_C100_CYY_20170406.jpg' 1] 1
['CZ_9005800284002_BBREV_C100_CYN_20180103.jpg' 0] 0
['CZ_9005800284002_BBREV_C100_CYY_20180103.jpg' 1] 1
['CZ_9005800284019_BBNEW_C100_CYN_20170406.jpg' 0] 0
['CZ_9005800284019_BBNEW_C100_CYY_20170406.jpg' 1] 1
['CZ_9005800284019_BBREV_C100_CYN_20180104.jpg' 0] 0
['CZ_9005800284019_BBREV_C100_CYN_20180129.jpg' 0] 0
['CZ_9005800284019_BBREV_C100_CYY_20180104.jpg' 1] 1
['CZ_9005800284019_BBREV_C100_CYY_20180129.jpg' 1] 1
['DE_3263280105224_00000_C100_CNN_20091012130605578.jpg' 0] 0
['DE_3574661351476_BBNEW_C100_CYN_20180123.jpg' 0] 0
['DE_3574661351476_BBNEW_C100_CYY_20180123.jpg' 1] 0
['DE_3574661352558_BBNEW_C100_CYN_20180123.jpg' 0] 0
['DE_3574661352558_BBNEW_C100_CYY_20180123.jpg' 1] 1
['DE_3574661393674_BBREV_C100_CYN_20180511.jpg' 0] 0
['DE_3574661393674_BBREV_C100_CYY_20180511.jpg' 1] 1
['DE_3607345265018_BBOLD_C100_CYN_20150615.jpg' 0] 0
['DE_3607345265018_BBOLD_C100_CYY_201

['FR_8410031936506_BBREV_C100_CYY_20190507.jpg' 1] 1
['FR_9049000000444_BBREV_C100_CYY_20151006.jpg' 1] 1
['FR_9049000000444_BBREV_C100_CYY_20151013.jpg' 1] 1
['FR_9049000000444_BBREV_C100_CYY_20151118.jpg' 1] 1
['FR_9049000000444_BBREV_C100_CYY_20180430.jpg' 1] 1
['GB_0000030173866_BBNEW_C100_CYY_20181220.jpg' 1] 1
['GB_0000054008021_BBNEW_C100_CYY_20180731.jpg' 1] 1
['GB_0000054008021_BBREV_C100_CYY_20180921.jpg' 1] 1
['GB_0000096133132_BBNEW_C100_CYY_20160630.jpg' 1] 1
['GB_0000096133132_BBREV_C100_CYY_20181106.jpg' 1] 1
['GB_0000096133132_BBREV_C100_CYY_20181207.jpg' 1] 1
['GB_0000096133132_BBREV_C100_CYY_20181219.jpg' 1] 1
['GB_0000096133149_BBNEW_C100_CYY_20160630.jpg' 1] 1
['GB_0000096133149_BBREV_C100_CYY_20181106.jpg' 1] 1
['GB_0000096133149_BBREV_C100_CYY_20181207.jpg' 1] 1
['GB_0000096133149_BBREV_C100_CYY_20181219.jpg' 1] 1
['GB_0000096176795_BBREV_C100_CYY_20181107.jpg' 1] 1
['GB_0000096176795_BBREV_C100_CYY_20181220.jpg' 1] 1
['GB_0000096176795_BBREV_C100_CYY_20190326.jpg

In [36]:
labels = None

In [112]:
data = pd.read_csv('test_names.csv')
data.columns = ['filename']
def display_images(upc):
    test_data = data[data.filename.str.contains(upc)]
    test_labels = cls.predict_generator(test_iterator(test_data,'Test'), steps=len(test_data))
    test_data['prediction'] = list(map(np.argmax,test_labels))
    labels = test_labels
    grid = HBox()
    boxes = []
    for row in test_data.values:
        img = HBox([Image(value=get_image(row[0], 'Test'), width=100,height=100)])
        if row[1] == 1:
            img.layout.justify_content = 'space-around'
            img.layout.border = '3px solid green'
        '''
        with open('correct.png' if row[1] == row[2] else 'wrong.png', 'rb') as f:
            R_W = Image(value=f.read(), format='png', width=20,height=20)
        '''
        boxes.append(VBox([img]))


    grid.children = boxes
    grid.layout.flex_flow = 'row wrap'
    grid.layout.justify_content = 'space-between'
    grid.layout.align_content = 'center'
    grid.layout.width = '80%'
    display(grid)
    return labels

In [116]:
l = display_images('0000011210006')

In [106]:
test_data = data[data.filename.str.contains('8712252683635')]

In [109]:
data = pd.read_csv('test_names.csv')
data.columns = ['filename']
testData = data[data.filename.str.contains('0070942123518')]
testData

,filename
0,AE_0070942123518_NPPFM_C100_C_2016100810543571...
1,AE_0070942123518_NPPFM_C100_C_2016100810543888...
2,AE_0070942123518_NPPFM_C100_C_2016100810544296...
3,AE_0070942123518_NPPFM_C100_C_2016100810544693...
4,AE_0070942123518_NPPFM_C100_C_2019050117132855...
5,AE_0070942123518_NPPFM_C100_C_2019050117133459...
6,AE_0070942123518_NPPFM_C100_C_2019050117133654...
7,AE_0070942123518_NPPFM_C100_C_2019050117134155...
5081,CH_0070942123518_00000_C080_CNN_20130723084215...
5082,CH_0070942123518_00000_C100_CNN_20130723084222...


In [107]:
test_data

,filename
29,DE_8712252683635_NPPFN_C100_C_2019052812491996...
30,DE_8712252683635_NPPFN_C100_C_2019052812492492...
31,DE_8712252683635_NPPFN_C100_C_2019052812492867...
32,DE_8712252683635_NPPFN_C100_C_2019052812493391...
22698,NL_8712252683635_NPPFN_C100_C_2019050613592715...
22699,NL_8712252683635_NPPFN_C100_C_2019050613593116...
22700,NL_8712252683635_NPPFN_C100_C_2019050613593719...
22701,NL_8712252683635_NPPFN_C100_C_2019050613593911...
